In [15]:
!pip install pexpect


ModuleNotFoundError: No module named 'pexpect'

In [ ]:
import asyncio
from playwright.async_api import async_playwright

FORM_URLS = [
    "https://kcdc.local/whitepaper/second-white-paper/",
    "https://kcdc.local/whitepaper/testing-me-out/",
]
DOWNLOAD_BASE = "https://kcdc.local/white-paper-download/"
TOTAL_USERS = 50


async def simulate_user(playwright, user_id, form_url):
    browser = await playwright.chromium.launch(headless=True)
    context = await browser.new_context()
    page = await context.new_page()

    try:
        await page.goto(form_url)
        await page.fill("#kcdc_name", f"Test User {user_id}")
        await page.fill("#kcdc_agency", f"Agency {user_id}")
        await page.fill("#kcdc_email", f"testuser{user_id}@example.com")
        await page.click(".kcdc-submit-button")
        await page.wait_for_url("**?success=true**", timeout=10000)

        final_url = page.url
        token = page.url.split("token=")[-1].split("&")[0]
        post_id = page.url.split("post_id=")[-1].split("&")[0]

        download_url = f"{DOWNLOAD_BASE}?action=kcdc_download_whitepaper&token={token}&post_id={post_id}"
        await page.goto(download_url)

        if await page.query_selector(".kcdc-download-error"):
            text = await page.inner_text(".kcdc-download-error")
            print(f"❌ User {user_id}: ERROR - {text.strip()}")
        elif await page.query_selector(".kcdc-download-documents a"):
            print(f"✅ User {user_id}: SUCCESS - Download ready")
        else:
            print(f"❌ User {user_id}: Unknown state")

    except Exception as e:
        print(f"❌ User {user_id}: Exception - {e}")
    finally:
        await context.close()
        await browser.close()


async def main():
    async with async_playwright() as playwright:
        tasks = []
        for i in range(TOTAL_USERS):
            form_url = FORM_URLS[i % len(FORM_URLS)]
            tasks.append(simulate_user(playwright, i + 1, form_url))
        await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(main())
